<p><font size="7" color='grey'> <b>
Anwendung Generativer KI
</b></font> </br></p>

<p><font size="6" color='grey'> <b>
Modul 11: LangChain: Agents
</b></font> </br></p>


---

# 1 | Übersicht
---

# 7.1: LangChain Agents

LangChain allows the use of [agents](https://python.langchain.com/v0.1/docs/modules/agents/), which revolutionize the way we interact with language models by transforming them into dynamic decision-makers. Unlike traditional chains, where each step is meticulously programmed, agents leverage the inherent reasoning capabilities of language models to choose their own paths. Imagine an intelligent system that not only understands the context but also determines the most effective sequence of actions to achieve a goal. This adaptability marks a significant departure from rigid, pre-defined workflows. By acting as a reasoning engine, the language model assesses each situation and decides which actions to take and in what order, offering unparalleled flexibility and efficiency. This chapter delves into the fascinating world of lang-chain agents, exploring how they harness the power of language models to autonomously navigate complex tasks, adapt to changing conditions, and deliver sophisticated solutions with minimal human intervention.

LangChain suppors multiple [agent types](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/). You will interface with these agents via two different methods:

* **Chat Models**: Designed to take in messages and output messages, requiring a specific prompting strategy for optimal performance.
* **LLMs**: Intended to take in a string and output a string, using a different prompting strategy to achieve the best results.

Some of the other important features for agent types include:

* **Supports Chat History**: Indicates if an agent can function as a chatbot, typically requiring advanced models.
* **Supports Multi-Input Tools**: Indicates if an agent can use tools requiring multiple inputs, often unsupported by earlier models.
* **Supports Parallel Function Calling**: Indicates if an agent can call multiple tools simultaneously, requiring more advanced models.

The LangChain agent types can be summarized as follows:

|Agent Type|Interface|Chat<br>History|Multi-Input<br>Tools|Parallel<br>Function Calling|When to use|
|--|--|--|--|--|--|
|Tool Calling|chat|✅|✅|✅|Used in this course, also tool-calling agents.
|XML|LLM|✅|||Use with XML models, such as Anthropic
|Structured Chat|chat|✅|✅||If you need to support tools with multiple inputs.
|JSON Chat|chat|✅|||If you are using a model good at JSON|
|ReAct|LLM|✅|||Complex reasoning|
|Self-Ask w/Search|LLM||||If you are using a simple model and only have one search tool|


In this course, we will focus primarily on tool-calling agents, which are designed to interact with external tools and APIs to perform specific tasks, such as data retrieval, processing, or executing actions based on user inputs. Tool-calling agents are highly versatile and are essential for building applications that require integration with various external systems. Besides tool-calling agents, there are two other important types: ReAct agents and Self-Ask with Search agents. ReAct agents combine reasoning and acting capabilities, making them suitable for complex decision-making and task automation scenarios where the agent needs to logically process information and take appropriate actions. Self-Ask with Search agents, on the other hand, are designed to enhance the language model's capabilities by integrating search functionalities, allowing the agent to retrieve up-to-date information from external sources. This makes them ideal for applications requiring accurate and current data retrieval, such as news aggregation, fact-checking, and research tools. Each type of agent serves a unique purpose, and understanding their use cases will enable you to select the appropriate agent for your specific application needs. We will also cover how to create an entirely custom agent.

## Key Components of Agents

Agents use a language model to decide on a sequence of actions. Unlike chains, where actions are predefined in the code, agents utilize a language model as a reasoning engine to determine the actions and their order.

* **AgentAction**: An AgentAction is a dataclass that defines the action an agent should take. It includes a tool property, specifying the name of the tool to be invoked, and a tool_input property, providing the input for that tool.

* **AgentFinish**: AgentFinish signifies the final result from an agent, ready to be returned to the user. It contains a return_values key-value mapping, typically holding an output key with a string representing the agent's response.

* **Intermediate Steps**: Intermediate steps represent previous agent actions and their corresponding outputs during the current run. These steps are crucial for informing future iterations, allowing the agent to track completed tasks. This is typed as a List (Tuple[AgentAction, Any), with observation being flexible and often a string.

## Agent Chain Structure

The agent is responsible for deciding the next step, usually powered by a language model, a prompt, and an output parser. Different agents use various prompting styles, input encoding methods, and output parsing techniques. For a comprehensive list of built-in agents, refer to the agent types section. Custom agents can also be created for more control.

* **Agent Inputs**: Inputs to an agent are key-value mappings, with intermediate_steps being the only required key, corresponding to the intermediate steps mentioned earlier. The PromptTemplate typically transforms these pairs into a format suitable for the LLM.

* **Agent Outputs**: The outputs can be the next action(s) to take or the final response to send to the user (AgentActions or AgentFinish). Concretely, this can be typed as Union (AgentAction, List[AgentAction], AgentFinish). The output parser converts the raw LLM output into one of these types.

## The AgentExecutor
The agent executor is the runtime that calls the agent, executes chosen actions, passes the action outputs back to the agent, and repeats the process. This runtime handles complexities such as:

* Selecting non-existent tools
* Managing tool errors
* Parsing tool invocation outputs
* Logging and observability at all levels (agent decisions, tool calls)

## Agent Tools
Tools are functions that an agent can invoke. The Tool abstraction consists of two components:

The input schema for the tool, informing the LLM of the necessary parameters.
The function to run, typically a Python function.

## Agent Toolkits
For common tasks, agents often require a set of related tools. LangChain provides toolkits—groups of 3-5 tools for specific objectives. For example, the GitHub toolkit includes tools for searching issues, reading files, and commenting. LangChain offers various toolkits to get started. For a full list, see the toolkits integrations section.

## Basic Agent Exmaple

Let's now take a look at a basic agent. An agent in this context is a system designed to perform specific tasks by leveraging tools and models. At a high level, this agent uses a language model to process inputs and make use of external tools to gather information. In our example, the agent is set up to answer questions using a search tool (DuckDuckGo) and a language model (OpenAI's ChatGPT). DuckDuckGo is used here because it is simple to integrate and does not require API keys. We will later explore how the agent is created and how it interacts with the tools and the language model to produce responses.

In [ ]:
from re import VERBOSE
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )

search_tool = DuckDuckGoSearchRun()

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [search_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What is the currnet value of the DJIA?"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'current value of the DJIA'}`


Dow Jones Industrial Average + Add to watchlist + Add an alert. DJI:DJI. Dow Jones Industrial Average. Actions. Add to watchlist; Add an alert; Price (USD) 39,411.21; Today's Change 260.88 / 0.67%; Shares traded 387.00m; ... Share price information may be rounded up/down and therefore not entirely accurate. Dow Jones Today: Get all information on the Dow Jones Index including historical chart, news and constituents. The Dow Jones Industrial Average ( DJINDICES:^DJI) is a stock index that tracks 30 of the largest U.S. companies. Created in 1896, it is one of the oldest stock indexes, and its performance is ... The observations for the Dow Jones Industrial Average represent the daily index value at market close. The market typically closes at 4 PM ET, except for holidays when it sometimes closes early. The Dow Jones Industrial Average provides a view of the US stock mark

{'input': 'What is the currnet value of the DJIA?',
 'output': 'The current value of the Dow Jones Industrial Average (DJIA) is 39,411.21.'}


The code sets up a basic agent to answer questions by integrating a language model and a search tool. Initially, it imports necessary modules and initializes the language model (ChatOpenAI) with specified parameters such as model type and temperature. The DuckDuckGoSearchRun tool is instantiated for performing web searches. A prompt template is pulled from a repository to guide the agent's responses. The agent is then created using the create_tool_calling_agent function, which combines the language model, tools, and prompt. Input data is provided to test the agent with a question about the president of the United States. The AgentExecutor class is used to manage the agent's execution, allowing it to process further queries, such as checking the current value of the DJIA, with verbosity enabled to provide detailed output during execution. This setup demonstrates the integration of a language model with a search tool to perform information retrieval and question-answering tasks.

# Module 7 Assignment

You can find the first assignment here: [assignment 7](https://github.com/jeffheaton/app_generative_ai/blob/main/assignments/assignment_yourname_t81_559_class7.ipynb)

# 7.2: LangChain Agent Tools


LangChain agents are versatile entities designed to perform specific tasks autonomously. Central to their functionality are [tools](https://python.langchain.com/v0.1/docs/modules/tools/), which are specialized components that agents can utilize to accomplish their objectives. These tools can range from data retrieval and processing utilities to interactive interfaces for user engagement. By leveraging these tools, LangChain agents can efficiently execute complex workflows, automate routine tasks, and provide intelligent solutions tailored to user needs. Whether it's querying databases, parsing documents, or interacting with APIs, the strategic use of tools enables LangChain agents to enhance productivity and deliver precise outcomes.


Large Language Models (LLMs) inherently lack access to real-time information such as the current date and time, stock market data, and breaking news. This limitation stems from their design, which relies on pre-existing datasets that do not include ongoing updates. Additionally, LLMs are not equipped to perform mathematical calculations directly, which can restrict their utility in scenarios requiring precise numerical operations. To overcome these constraints, LLMs can employ specialized tools. For instance, search engine tools enable LLMs to retrieve the latest information from the web, ensuring up-to-date responses. Similarly, tools designed for mathematical computation can assist LLMs in accurately processing and solving mathematical problems, thereby enhancing their overall capability and accuracy.

* [Available Langchain Tools](https://python.langchain.com/v0.2/docs/integrations/tools/)
* [Available Langchain Toolkits](https://python.langchain.com/v0.2/docs/integrations/toolkits/)

## Tools for Math


We will begin our exploration of LangChain tools by using a tool specifically designed for math. Large Language Models (LLMs), like ChatGPT, are generally poor at performing mathematical calculations without the assistance of specialized tools. This is because LLMs are trained primarily on textual data and lack the precision required for accurate arithmetic operations. Consequently, they often produce inaccurate results when asked to perform math independently. To see how a LLM actually performs mathematics, I asked ChatGPT and it gave a decent high-level summary.

> Large Language Models (LLMs), like me, do not inherently perform arithmetic calculations the same way a calculator or dedicated algorithm would. Instead, we generate responses based on patterns in the data we've been trained on. Here's a simplified explanation of how we handle such tasks:
>
> Pattern Recognition: During training, LLMs are exposed to vast amounts of text data, which includes examples of arithmetic and mathematical reasoning. We learn patterns and structures in these examples, enabling us to approximate calculations.
>
> Token Prediction: When asked to perform a calculation, an LLM doesn't actually "calculate" in the traditional sense. Instead, it predicts the most likely sequence of tokens (numbers, in this case) that should follow based on the input. This prediction is influenced by the training data but does not involve real arithmetic operations.
>
> Approximation and Heuristics: For smaller or simpler calculations, the model might generate the correct answer because it has seen enough examples during training. For larger or more complex calculations, the model might generate an approximate answer or even make a guess based on learned patterns.
>
> For example, if you ask an LLM to multiply 872947493 by 7492374932, it will try to generate a plausible sequence of digits based on what it has seen in the training data, but this sequence is unlikely to be correct without an actual computational algorithm.
>
>Here’s a brief comparison of how a traditional method (e.g., a calculator or algorithm) and an LLM approach such a problem:
>
> * Traditional Method: Uses precise algorithms to perform each step of the multiplication (e.g., long multiplication or fast algorithms like the Karatsuba algorithm).
> * LLM Method: Predicts the next sequence of digits based on patterns and probabilities from the training data.
So, while an LLM might "attempt" to give an answer, it lacks the precision and algorithmic foundation to guarantee accuracy for complex arithmetic without dedicated computational tools.

To see this in action, let's ask an LLM to perform a mathematical operation. We will choose numbers that were unlikely in the LLM's training data.



In [ ]:
from langchain_openai import ChatOpenAI

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )

print(llm.invoke("What is 8273 times 1821?"))

content='8273 times 1821 equals 15,086,213.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 17, 'total_tokens': 31}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None} id='run-3a675778-1103-4384-8350-8082c63cca8e-0' usage_metadata={'input_tokens': 17, 'output_tokens': 14, 'total_tokens': 31}


The resulting number appears reasonable, but that's the point. LLMs are trained to produce believable results, not necessarily correct ones. To verify the LLM, we'll have Python perform this calculation.

In [ ]:
print(8273 * 1821)

15065133


In [ ]:
print( abs(15065133 - 15055433 ) )


9700


We can see that the LLM was several thousand off. Unfortunately, LangChain does not include a calculator tool, at least as of summer 2024. We will look at two approaches. First, we will use the LangChain-provided PythonREPL. The following code shows how to use PythonREPL.

In [ ]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()

python_repl.run("print(1+1)")

'2\n'

It begins by importing necessary modules and classes from the LangChain, LangChain OpenAI, and LangChain Community libraries.
A tool for executing Python commands (repl_tool) is created using the Tool class, with a description indicating its purpose as a Python shell that requires valid Python commands. The function python_repl.run is assigned to execute the commands.

The prompt for the agent is pulled from a hub, specified by the identifier "hwchase17/openai-functions-agent".

An agent is then created using the create_tool_calling_agent function, which takes the language model (llm), the tools (here, just repl_tool), and the prompt. This agent is wrapped in an AgentExecutor with the verbose mode enabled to provide detailed logs of the execution process.
Finally, the agent is invoked with an input command to calculate the product of 8273 and 1821.

In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain.tools import StructuredTool
from pydantic import BaseModel

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
    model=MODEL,
    temperature=0.2,
    n=1
)

# Define a Pydantic schema for the input arguments
class PythonReplInput(BaseModel):
    input: str

# Define the tool using StructuredTool with an args_schema
def run_python_code(input: str, **kwargs):
    try:
        # Safely evaluate the input string as a Python expression
        result = eval(input)
        return str(result)
    except Exception as e:
        return str(e)

repl_tool = StructuredTool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=run_python_code,  # Use the custom Python REPL function
    args_schema=PythonReplInput  # Define the expected input schema
)

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [repl_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Pass a string Python command as input
result = agent_executor.invoke({"input": "What is 8273 * 1821?"})

print(result)


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:5515: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))




> Entering new AgentExecutor chain...

Invoking: `python_repl` with `{'input': '8273 * 1821'}`


15065133The result of \( 8273 \times 1821 \) is \( 15,065,133 \).

> Finished chain.
{'input': 'What is 8273 * 1821?', 'output': 'The result of \\( 8273 \\times 1821 \\) is \\( 15,065,133 \\).'}


## Create a Cusom Math Tool

The Python REPL tool we just used can execute any Python command. Therefore, it can be a security concern if we only wish to perform math calculations. In this section, we will see how to create a custom tool that can only perform basic math calculations.

In [ ]:
from langchain_openai import OpenAI
import numpy as np
from langchain.chains.base import Chain

class SafeCalculator:
    def calculate(self, expression):
        try:
            # Evaluate the mathematical expression using NumPy
            result = eval(expression, {"__builtins__": None}, {"np": np})
            return result
        except Exception as e:
            return str(e)

class CalculatorChain(Chain):
    calculator: SafeCalculator

    def _call(self, inputs):
        expression = inputs["expression"]
        result = self.calculator.calculate(expression)
        return {"result": result}

    @property
    def input_keys(self):
        return ["expression"]

    @property
    def output_keys(self):
        return ["result"]

# Initialize the safe calculator tool
safe_calculator = SafeCalculator()

# Example usage
chain = CalculatorChain(calculator=safe_calculator)
expression = "3 * (2 + 5) / 7"
inputs = {"expression": expression}
result = chain(inputs)
print(f"Result: {result['result']}")


Result: 3.0


<ipython-input-7-34b3ab86af62>:38: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = chain(inputs)


In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain.tools import StructuredTool
from pydantic import BaseModel

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
    model=MODEL,
    temperature=0.2,
    n=1
)

# Define a Pydantic schema for the input arguments
class MathExpressionInput(BaseModel):
    input: str

# Define a safe calculator function
def safe_calculator(input: str, **kwargs):
    try:
        # Safely evaluate the input math expression
        result = eval(input)
        return str(result)
    except Exception as e:
        return str(e)

# Define the tool using StructuredTool with args_schema
safe_math_tool = StructuredTool(
    name="safe_calc",
    description="A math calculator used to evaluate mathematical expressions. Input should be a valid math expression, similar to Python.",
    func=safe_calculator,  # Use the safe calculator function
    args_schema=MathExpressionInput  # Define the expected input schema
)

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [safe_math_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Pass a mathematical expression as input
result = agent_executor.invoke({"input": "8273 * 1821"})

print(result)


/usr/local/lib/python3.10/dist-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `safe_calc` with `{'input': '8273 * 1821'}`


15065133The result of \( 8273 \times 1821 \) is \( 15,065,133 \).

> Finished chain.
{'input': '8273 * 1821', 'output': 'The result of \\( 8273 \\times 1821 \\) is \\( 15,065,133 \\).'}


# 7.3: LangChain-Abruf- und Suchtools

In diesem Abschnitt werden wir uns zwei Arten der Abfrage für die Verwendung mit LangChain-Agenten ansehen: Such- und Abfragetools. Suchtools ermöglichen Ihrem Agenten den Zugriff auf Suchmaschinen wie Google, um dem Agenten aktuelle Informationen bereitzustellen. Darüber hinaus können Sie RAG verwenden, um das Wissen Ihres Agenten mit einem vektorisierten Dokumentspeicher zu erweitern, ähnlich wie bei früheren RAG-Beispielen in diesem Kurs.

## Nutzung der Duck Duck Go-Suche

DuckDuckGo ist eine Suchmaschine, die mit einem starken Fokus auf die Privatsphäre der Benutzer entwickelt wurde. Im Gegensatz zu vielen herkömmlichen Suchmaschinen verfolgt DuckDuckGo Ihre Suchvorgänge nicht und speichert keine persönlichen Daten, wodurch ein sichereres und privateres Online-Erlebnis gewährleistet wird. Auch wenn Sie vielleicht noch nie von DuckDuckGo gehört haben, erfreut es sich aufgrund seines unkomplizierten Suchansatzes und seines Engagements für die Privatsphäre der Benutzer großer Beliebtheit.

Für viele Beispiele in diesem Kurs werden wir [DuckDuckGo](https://duckduckgo.com/) aus mehreren wichtigen Gründen verwenden. Erstens ist die API kostenlos zugänglich und erfordert keinen Authentifizierungsschlüssel, was den Prozess vereinfacht und es uns ermöglicht, direkt mit dem Programmieren zu beginnen, ohne uns mit komplexen Setups herumschlagen zu müssen. Zweitens hilft uns die Verwendung von DuckDuckGo dabei, Datenschutzprobleme im Auge zu behalten und die Bedeutung des Schutzes personenbezogener Daten bei unseren digitalen Interaktionen zu verstehen. Diese praktische Erfahrung wird nicht nur unsere technischen Fähigkeiten verbessern, sondern auch unser Wissen über die vielfältigen Tools erweitern, die im digitalen Ökosystem verfügbar sind.

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
search_tool.run("Who is the president of the USA?")

'Joe Biden (born November 20, 1942, Scranton, Pennsylvania, U.S.) 46th president of the United States (2021- ) and 47th vice president of the United States (2009-17) in the Democratic administration of Pres. Barack Obama. He previously represented Delaware in the U.S. Senate (1973-2009). Basic Qualifications: Age, Citizenship and Residency. According to the Constitution, there are just three basic qualifications for being president: Age: The person must be at least 35 years old. Former President Donald Trump and President Joe Biden faced off on Thursday night during the first presidential debate of the 2024 race for the White House. The CNN Presidential Debate marks the ... The president of the United States is the: U.S. head of state; Chief executive of the federal government; Commander-in-Chief of the armed forces; Current president. The 46th and current president of the United States is Joseph R. Biden, Jr. He was sworn into office on January 20, 2021. Former U.S. presidents. The Un

In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )

search_tool = DuckDuckGoSearchRun()

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [search_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "Return the value of the DJIA as a floating point number, return just the number, no text or comments."})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'current value of DJIA'}`


Dow Jones Industrial Average + Add to watchlist + Add an alert. DJI:DJI. Dow Jones Industrial Average. Actions. Add to watchlist; Add an alert; Price (USD) 39,411.21; Today's Change 260.88 / 0.67%; Shares traded 387.00m; ... Share price information may be rounded up/down and therefore not entirely accurate. Dow Jones Today: Get all information on the Dow Jones Index including historical chart, news and constituents. The Dow Jones Industrial Average ( DJINDICES:^DJI) is a stock index that tracks 30 of the largest U.S. companies. Created in 1896, it is one of the oldest stock indexes, and its performance is ... Headlines for Dow Jones Industrial Average (Dow Jones Global:DJIA) News for Dow Jones Industrial Average. Thursday, June 27, 2024. 04:44 PM ET. Hedge funds are dumping tech stocks despite their recent rally, says Goldman Sachs MarketWatch. ... Historical and current e

{'input': 'Return the value of the DJIA as a floating point number, return just the number, no text or comments.',
 'output': '39411.21'}

## Verwenden der Tavily-Suche

Wir werden nun untersuchen, wie man [Tavily](https://tavily.com/) mit LangChain verwendet. Während sich dieser Kurs aufgrund des kostenlosen Zugriffs hauptsächlich auf die Verwendung von DuckDuckGo konzentriert, ist es wichtig zu beachten, dass Tavily auch eine kostenlose Version anbietet. Der kostenlose Plan von Tavily ermöglicht es Benutzern, bis zu 1.000 API-Aufrufe pro Monat durchzuführen, ohne dass eine Kreditkarte erforderlich ist. Damit ist es für diejenigen zugänglich, die die erweiterten Suchfunktionen nutzen möchten, die auf große Sprachmodelle (LLMs) zugeschnitten sind.

Die Verwendung von Tavily kann mehrere Vorteile bieten. Erstens ist Tavily für LLMs optimiert und liefert hochrelevante, sachliche und kontextbezogen passende Suchergebnisse. Diese Optimierung trägt dazu bei, Halluzinationen zu reduzieren und die allgemeinen Entscheidungsfähigkeiten von KI-Agenten zu verbessern. Zweitens unterstützt Tavily die Retrieval-Augmented Generation (RAG), die für die Gewährleistung der Genauigkeit und Relevanz der von LLMs verwendeten Informationen von entscheidender Bedeutung ist.

Darüber hinaus sorgt die nahtlose Integration von Tavily in beliebte KI-Frameworks wie LangChain und LlamaIndex dafür, dass Entwickler es problemlos in ihre bestehenden Arbeitsabläufe integrieren können. Die Effizienz und Geschwindigkeit der Suchfunktionen von Tavily, kombiniert mit seiner umfassenden Abdeckung durch die Aggregation von Daten aus mehreren Quellen, machen es zu einem unschätzbaren Werkzeug für KI-gesteuerte Anwendungen. Während DuckDuckGo für viele Benutzer eine praktische und kostengünstige Wahl bleibt, kann das Ausprobieren der erweiterten Funktionen von Tavily die Leistung und Genauigkeit Ihrer KI-Modelle erheblich verbessern.

Der folgende Code zeigt eine Beispielsuchabfrage, die Tavily präsentiert wird. Der LLM würde diese Ergebnisse verarbeiten, um zusätzliche Informationen für die Bildung einer Antwort zu erhalten.




In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults()
tool.invoke({"query": "What happened in the latest burning man floods"})

[{'url': 'https://www.today.com/news/what-is-burning-man-flood-death-rcna103231',
  'content': '"\nAn update on BurningMan.org posted the night of Sept. 3 offered those still on site a "Wet Playa Survival Guide" with tips, as well as an update saying that conditions were "improving" and that roads in Black Rock City were expected to open on Sept. 4 for "Exodus."\n"Thank goodness this community knows how to take care of each other," the Instagram page for Burning Man Information Radio wrote on a post predicting more rain.\nNeal Katyal, Chris Rock and others detail their ‘escape’ from Black Rock City\nComedian Chris Rock and Diplo, producer and DJ,\xa0were among this year\'s Burning Man attendees. Why are people stranded at Burning Man?\nEach year participants in the Burning Man event gather for nine days in an "annual experiment in temporary community dedicated to Radical Self-expression and Radical Self-reliance," according to burningman.org.\nHeld in Nevada since 1991, the festival se

Wir sehen jetzt, dass wir es genau so als Tool hinzufügen können, wie wir DuckDuckGo verwendet haben.

In [ ]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor

search_tool = TavilySearchResults()

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [search_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "Who is the oldest world leader?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'oldest world leader 2021'}`


[{'url': 'https://www.pewresearch.org/short-reads/2024/05/01/as-biden-and-trump-seek-reelection-who-are-the-oldest-and-youngest-current-world-leaders/', 'content': "Joe Biden, at 81, is the oldest American president, a distinction he's held since entering office at age 78. As Biden runs for reelection in 2024, he is the ninth oldest national leader in the world, according to a Pew Research Center analysis of sitting leaders in 187 United Nations member states. Former President Donald Trump, who is ..."}, {'url': 'https://en.wikipedia.org/wiki/List_of_oldest_living_state_leaders', 'content': 'The oldest living former state leader is Guillermo Rodríguez of Ecuador at the age of 100 years, 236 days. Leaders currently in office are in bold in green, with Paul Biya of Cameroon being the oldest currently serving head of state. Gallery. ... 11 Jan 2021: 12:'}, {'url':

{'input': 'Who is the oldest world leader?',
 'output': 'The oldest world leader in 2021 is Paul Biya of Cameroon.'}

## Verwenden von Agenten mit Abruf

Wir können die Abfrage auch für Agenten nutzen. Wir werden das RAG-System verwenden, das wir zuvor verwendet haben, um zufällig generierte biografische Skizzen für Personen zu erstellen, die bei fünf fiktiven Unternehmen arbeiten. Wir verwenden dieselben Schritte wie zuvor, um diese Dokumente in einen LangChain-Retriever zu laden.

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain import OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import display_markdown
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.inmemory import InMemoryVectorStore
from langchain.schema import Document
import requests

urls = [
    "https://data.heatonresearch.com/data/t81-559/bios/DD.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/FT.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/GS.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/NGS.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/TI.txt"
]

def chunk_text(text, chunk_size, overlap):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

chunk_size = 900
overlap = 300

documents = []

for url in urls:
print(f"Lesen: {url}")
    response = requests.get(url)
    response.raise_for_status()  # Stellen Sie sicher, dass uns schlechte Antworten auffallen
    content = response.text
    chunks = chunk_text(content, chunk_size, overlap)
    for chunk in chunks:
        document = Document(page_content=chunk)
        documents.append(document)

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.vectorstores import Chroma

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()

Reading: https://data.heatonresearch.com/data/t81-559/bios/DD.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/FT.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/GS.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/NGS.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/TI.txt


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Wir testen, ob der Retriever Text finden kann, der mit einer der betrügerischen Mitarbeiterinnen, Samantha Doyle, verknüpft ist.

In [ ]:
retriever.invoke("who is Samantha Doyle")[0]

Document(page_content='the next generation of female tech leaders.\n\nSamantha Doyle is a seasoned Project Manager at Global Solutions, an innovative tech company known for pioneering smart city technologies. With over a decade of experience in the tech industry, Samantha has played a pivotal role in leading her team through successful launches of multiple high-profile sustainability projects. She holds a Masterâ\x80\x99s degree in Systems Engineering from MIT and has a passion for integrating eco-friendly practices into urban development. Outside of her professional life, Samantha is an avid rock climancer and enjoys mentoring young women interested in STEM careers, often volunteering her time at local high schools and community centers. Her dedication to both her career and community has made her a respected leader at Global Solutions and an influential figure in her field.\n\nSamantha Clarke is a seasoned Project Ma')

Wir konstruieren nun das Tool und geben dem Agenten, wie Sie sehen, auch Anweisungen, wann er diesen Retriever verwenden soll.

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about people who work for several companies. For any questions about people you do not know, you must use this tool!",
)

Wir können dem Agenten nun eine Frage stellen, die sowohl die Suche als auch die RAG-Dokumente verwendet. Die folgende Frage:

> Wie sind die Berufsaussichten für Samantha Doyles Karriere im Jahr 2024? Senden Sie uns eine zwei Sätze umfassende Einschätzung ihrer beruflichen Zukunft.

Diese Frage erfordert die RAG-Daten, um Samanthas Job zu bestimmen. Anschließend wird die Suchmaschine verwendet, um die aktuellen Aussichten für diesen Job im Jahr 2024 zu finden.

In [ ]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor

MODEL = 'gpt-4o-mini'

search_tool = search_tool = DuckDuckGoSearchRun()

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [search_tool, retriever_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What are the job prospects in 2024 for Samantha Doyle's career? Return just a 2-sentence assessment of her career future."})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'job prospects for Samantha Doyle in 2024'}`


men interested in STEM careers, aiming to inspire the next generation of female tech leaders.

Samantha Doyle is a seasoned Project Manager at Global Solutions, an innovative tech company known for pioneering smart city technologies. With over a decade of experience in the tech industry, Samantha has been instrumental in leading her team through several high-profile international projects, significantly enhancing urban infrastructure and sustainability in cities across Europe and Asia. A graduate of MIT with a degree in Systems Engineering, her expertise lies in integrating complex systems with emerging technologies to create seamless, efficient solutions. Outside of her professional life, Samantha is an avid rock climber and volunteers her time mentoring young women interested in STEM careers, aiming to inspire the next generation of female tech leaders.

{'input': "What are the job prospects in 2024 for Samantha Doyle's career? Return just a 2-sentence assessment of her career future.",
 'output': 'Samantha Doyle, a seasoned Project Manager at Global Solutions, is well-positioned for a successful career future in 2024. With over a decade of experience in leading high-profile international projects and expertise in integrating complex systems with emerging technologies, her innovative approach and leadership skills are likely to continue driving her success in the tech industry. Additionally, her dedication to mentoring young women in STEM careers showcases her commitment to inspiring the next generation of female tech leaders, further solidifying her impact and influence in the field.'}

# 7.4: Erstellen von LangChain-Agenten


In diesem Kapitel stellen wir ein umfassendes Modul vor, das die Fähigkeiten eines vollständigen LangChain-Agenten mit Websuch-, Abfrage- und Speicherfunktionen demonstriert. Dieser Agent ist darauf ausgelegt, Fragen zu beantworten, indem er die aktuellste Dokumentation zu LangChain durch Retrieval-Augmented Generation (RAG) nutzt und die Grenzen des Wissens des Basismodells überschreitet. Durch die Integration von Websuche und -abruf in Echtzeit kann der Agent auf die neuesten Informationen zugreifen und diese integrieren, wodurch sichergestellt wird, dass seine Antworten genau und aktuell sind. Darüber hinaus ermöglicht die Speicherkomponente dem Agenten, relevante Informationen zu behalten und abzurufen, wodurch seine Fähigkeit verbessert wird, fundierte und kontextrelevante Antworten zu geben. In diesem Modul lernen die Leser, wie man einen intelligenten Agenten erstellt, der nicht nur Anfragen versteht und verarbeitet, sondern auch seine Wissensdatenbank kontinuierlich aktualisiert, um mit den neuesten Fortschritten in LangChain Schritt zu halten.

Wir beginnen mit dem Importieren der für unseren LangChain-Agenten erforderlichen Bibliotheken und Module, einschließlich derer für die Websuche, Abfrage und Speicherverwaltung. Genau wie in den vorherigen Kapiteln erstellen wir ein OpenAI-Sprachmodell (LLM), das als Grundlage für unseren Agenten dient. Dieses Modell wird für die Interpretation und Generierung von menschenähnlichem Text auf der Grundlage der empfangenen Eingaben von entscheidender Bedeutung sein. Durch die Nutzung des OpenAI-LLM stellen wir sicher, dass unser Agent über eine robuste und vielseitige Sprachverständnisfähigkeit verfügt, die durch die Integration von Echtzeit-Websuch- und -abruffunktionen noch weiter verbessert wird. Dieses Setup ermöglicht es dem Agenten, auf die aktuellsten und relevantesten Informationen zuzugreifen und diese zu nutzen und umfassende und genaue Antworten auf Benutzeranfragen zu liefern.


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )


In diesem Abschnitt verwenden wir Retrieval-Augmented Generation (RAG), um auf die neueste Version der LangChain-Dokumentation zuzugreifen, die aktueller ist als die Daten, mit denen das grundlegende Modell ursprünglich trainiert wurde. Wir beginnen mit der Erstellung einer WebBaseLoader-Instanz, um Inhalte von der URL https://docs.smith.langchain.com/overview zu laden und stellen so sicher, dass wir mit der aktuellsten Dokumentation arbeiten. Sobald der Inhalt geladen ist, verwenden wir RecursiveCharacterTextSplitter, um den Text in kleinere, handhabbare Blöcke von jeweils 1000 Zeichen aufzuteilen, mit einer Überlappung von 200 Zeichen, um den Kontext beizubehalten. Diese Blöcke werden dann mit OpenAIEmbeddings() in Einbettungen umgewandelt und wir indizieren diese Einbettungen mit FAISS (Facebook AI Similarity Search), um eine effiziente Ähnlichkeitssuche durchzuführen. Schließlich erstellen wir einen Retriever aus dem FAISS-Index, der es unserem LangChain-Agenten ermöglicht, die Dokumenteinbettungen abzufragen und die relevantesten Dokumentationsblöcke als Antwort auf Benutzerfragen abzurufen. Dieser Ansatz stellt sicher, dass unser Agent die neuesten Informationen nutzen kann, was seine Fähigkeit verbessert, genaue und aktuelle Antworten zu geben.

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

Wir senden nun eine einfache Abfrage an den Retriever, um zu sehen, welche Daten er zur Beantwortung der Frage zurückgibt. Die zurückgegebenen Daten stammen von der Website und sind noch nicht von einem LLM verarbeitet worden.

In [ ]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";co

Um Zugriff auf die gerade heruntergeladenen Webdaten zu ermöglichen, verwenden wir ein Agent-Retriever-Tool. Dieses Tool wird erstellt, indem der zuvor erstellte Retriever mit der Klasse VectorStoreRetrieverTool umschlossen wird, wobei als Name „Documentation“ und als Zweck eine kurze Beschreibung angegeben wird. Durch die Integration dieses Agent-Retriever-Tools kann unser LangChain-Agent die Webdaten effektiv zum Generieren von Antworten nutzen. Der ZeroShotAgent wird dann mit diesem Tool konfiguriert und der Agent mit AgentExecutor initialisiert. Dieses Setup ermöglicht es unserem Agenten, die relevantesten Informationen aus der neuesten LangChain-Dokumentation abzurufen und zu verarbeiten, und stellt sicher, dass er Anfragen genau und umfassend beantworten kann.

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

Wir demonstrieren auch eine Abfrage für die Tavily-Suche. Dadurch können wir die heruntergeladenen Daten mit Ad-hoc-Websuchen erweitern, um sowohl das Basismodell als auch die RAG-Datenquelle zu verbessern.

In [ ]:
search = TavilySearchResults()

search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1719777546, 'localtime': '2024-06-30 12:59'}, 'current': {'last_updated_epoch': 1719776700, 'last_updated': '2024-06-30 12:45', 'temp_c': 22.2, 'temp_f': 72.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 270, 'wind_dir': 'W', 'pressure_mb': 1016.0, 'pressure_in': 29.99, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 55, 'cloud': 0, 'feelslike_c': 24.6, 'feelslike_f': 76.4, 'windchill_c': 17.4, 'windchill_f': 63.3, 'heatindex_c': 17.4, 'heatindex_f': 63.3, 'dewpoint_c': 11.9, 'dewpoint_f': 53.3, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 'gust_mph': 18.1, 'gust_kph': 29.2}}"},
 {'url': 'https://www.msn.com/en-us/weather/topstori

Als Nächstes richten wir den LangChain-Agenten ein, indem wir verschiedene Tools integrieren und den Agenten so konfigurieren, dass er sie effektiv nutzen kann. Wir beginnen mit der Definition einer Liste von Tools, die unser Websuchtool und das zuvor erstellte Retriever-Tool enthält. Als Nächstes ziehen wir mit hub.pull("hwchase17/openai-functions-agent") eine vordefinierte Eingabeaufforderung aus dem LangChain-Hub, die strukturierte Nachrichten bereitstellt, die die Interaktionen des Agenten leiten. Mit dieser Eingabeaufforderung erstellen wir mit der Funktion create_tool_calling_agent einen Tool-aufrufenden Agenten und übergeben dabei unser Sprachmodell (LLM), die Liste der Tools und die Eingabeaufforderung. Schließlich initialisieren wir den Agent-Executor mit AgentExecutor, geben den Agenten und die Tools an und aktivieren den ausführlichen Modus für eine detaillierte Protokollierung. Diese Einrichtung stellt sicher, dass unser LangChain-Agent gut ausgestattet ist, um sowohl die Websuch- als auch die Retrieval-Funktionen zu nutzen und genaue und aktuelle Antworten bereitzustellen.

In [ ]:
tools = [search, retriever_tool]

# Holen Sie sich die zu verwendende Eingabeaufforderung – Sie können diese ändern!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Wir können sehen, wie der Agent auf eine einfache, triviale Eingabeaufforderung reagiert.

In [ ]:
agent_executor.invoke({"input": "hi!"})





> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

Als nächstes stellen wir ihm eine Frage zu Langsmith. Wir sehen, dass der Agent mehrere Tools verwendet.

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing by providing a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, ensuring that you can ship quickly and with confidence. LangSmith works independently, so the use of LangChain is not necessary. Here are some steps to get started with LangSmith for testing:\n\n1. Install LangSmith using Python or TypeScript:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on Create API Key.\n\n3. Set up your environment:\n   - Export the necessary environment variables like `LANGCHAIN_TRACING_V2`, `LANGCHAIN_API_KEY`, and optionally `OPENAI_API_KEY`.\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces. You can log your first trace to start testing your application.\n\nLangSmith also 

Wir können ihm auch eine Frage zu aktuellen Ereignissen stellen, woraufhin der Agent zum Suchtool geht.

In [ ]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.cbsnews.com/philadelphia/news/pa-weather-today-june-8-2024-philadelphia-jersey-shore/', 'content': "Watch CBS News Beautiful, sunny weather around Philadelphia and Jersey Shore Saturday, rain could return Sunday By Tammie Souza June 8, 2024 / 9:43 AM EDT / CBS Philadelphia PHILADELPHIA (CBS) -- High pressure is in control of the weather here in the Philadelphia area, and it will be a sunny, wonderful day with a light breeze and a high of 83 degrees in the city.  We hit 88 degrees by Thursday and could be passing 90 degrees on Friday and the heat dome over the western U.S. slides east toward us at the end of the week.  Friday could be the start of a stretch of 90-degree days in Philadelphia, even though we haven't even reached the start of astronomical summer yet - that's on June 20 at 4:51 p.m.  68 NEXT Weather Radars Hourly Forecast First

{'input': 'whats the weather in sf?',
 'output': 'The weather in San Francisco is currently in the low 60s in Daly City, Pacifica, and Half Moon Bay, mid-to upper 60s in South San Francisco and San Bruno, and the mid-to upper 70s in San Mateo and Redwood City. There is a threat of thunderstorms in the North Bay, East Bay, and Sacramento Valley during the morning and early afternoon, with the threat shifting northeastward to the Sierra Nevada by the evening.'}

## Chatverlauf/Speicher hinzufügen

Um die Fähigkeiten unseres LangChain-Agenten zu verbessern, fügen wir einen Chat-Speicher hinzu, damit er sich an frühere Interaktionen erinnern und den Kontext während einer Konversation beibehalten kann. Diese Ergänzung ermöglicht es dem Agenten, kohärentere und kontextbezogenere Antworten zu geben. Wir beginnen mit dem Importieren der erforderlichen Module zum Verwalten von Chat-Nachrichtenverläufen. Dann erstellen wir eine Instanz von ChatMessageHistory, um die während der Konversation ausgetauschten Nachrichten zu speichern. Indem wir unseren vorhandenen Agent-Executor mit RunnableWithMessageHistory umschließen, integrieren wir die Chat-Speicherfunktionalität. Dieses Setup stellt sicher, dass jede Sitzung einen Nachrichtenverlauf beibehalten kann, was reibungslosere und natürlichere Interaktionen mit dem Agenten ermöglicht. Der folgende Codeausschnitt veranschaulicht diese Implementierung:

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # Dies ist erforderlich, da in den meisten realen Szenarien eine Sitzungs-ID erforderlich ist
    # Es wird hier nicht wirklich verwendet, da wir eine einfache im Speicher befindliche ChatMessageHistory verwenden.
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

Wir werden jetzt den Speicher testen. Ich beginne damit, mich vorzustellen.

In [ ]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm Jeff"},
    # Dies ist erforderlich, da in den meisten realen Szenarien eine Sitzungs-ID erforderlich ist
    # Es wird hier nicht wirklich verwendet, da wir eine einfache im Speicher befindliche ChatMessageHistory verwenden.
    config={"configurable": {"session_id": "x123"}},
)



> Entering new AgentExecutor chain...
Hello Jeff! How can I assist you today?

> Finished chain.


{'input': "hi! I'm Jeff",
 'chat_history': [],
 'output': 'Hello Jeff! How can I assist you today?'}

Als nächstes stelle ich ihm eine Frage und wir sehen, dass es sich daran erinnert, wer ich bin.

In [ ]:
agent_with_chat_history.invoke(
    {"input": "how can LangSmith help with testing"},
    # Dies ist erforderlich, da in den meisten realen Szenarien eine Sitzungs-ID erforderlich ist
    # Es wird hier nicht wirklich verwendet, da wir eine einfache im Speicher befindliche ChatMessageHistory verwenden.
    config={"configurable": {"session_id": "x123"}},
)



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY

{'input': 'how can LangSmith help with testing',
 'chat_history': [HumanMessage(content="hi! I'm Jeff"),
  AIMessage(content='Hello Jeff! How can I assist you today?')],
 'output': 'LangSmith can help with testing by providing a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. Here are some steps to get started with LangSmith for testing:\n\n1. Install LangSmith:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on Create API Key.\n\n3. Set up your environment:\n   - Export the necessary environment variables like `LANGCHAIN_TRACING_V2` and `LANGCHAIN_API_KEY`.\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces. You can log your first trace to start testing.\n\nBy following these steps, you can leverage LangS

Wir können den Speicher weiter überprüfen und testen.

In [ ]:
agent_with_chat_history.invoke(
    {"input": "Who am I? What did I just ask you about?!"},
    # Dies ist erforderlich, da in den meisten realen Szenarien eine Sitzungs-ID erforderlich ist
    # Es wird hier nicht wirklich verwendet, da wir eine einfache im Speicher befindliche ChatMessageHistory verwenden.
    config={"configurable": {"session_id": "x123"}},
)



> Entering new AgentExecutor chain...
You are Jeff, and you just asked me about how LangSmith can help with testing. If you have any more questions or need further assistance, feel free to let me know!

> Finished chain.


{'input': 'Who am I? What did I just ask you about?!',
 'chat_history': [HumanMessage(content="hi! I'm Jeff"),
  AIMessage(content='Hello Jeff! How can I assist you today?'),
  HumanMessage(content='how can LangSmith help with testing'),
  AIMessage(content='LangSmith can help with testing by providing a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. Here are some steps to get started with LangSmith for testing:\n\n1. Install LangSmith:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on Create API Key.\n\n3. Set up your environment:\n   - Export the necessary environment variables like `LANGCHAIN_TRACING_V2` and `LANGCHAIN_API_KEY`.\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces. You can log your firs

# 7.5: Benutzerdefinierte LangChain-Agenten

In diesem Abschnitt befassen wir uns mit der Erstellung von benutzerdefinierten Agenten in LangChain. Durch die Entwicklung maßgeschneiderter Tools können wir die Funktionalität unserer Agenten verbessern, um komplexere Aufgaben zu bewältigen. Diese benutzerdefinierten Agenten ermöglichen es uns, Lösungen an spezifische Anforderungen anzupassen und so anspruchsvollere Interaktionen und Problemlösungsfunktionen zu ermöglichen. Durch die Nutzung des flexiblen Frameworks von LangChain können wir diese benutzerdefinierten Tools nahtlos integrieren, um einzigartige Herausforderungen zu bewältigen und die Leistung zu optimieren und so eine robustere und anpassungsfähigere Lösung für komplexe Szenarien bereitzustellen.

## Ein einfaches benutzerdefiniertes Tool

Wir beginnen mit einem sehr einfachen Tool: einem Knopf, den das Sprachmodell (LLM) drücken kann. Indem wir diesen Knopf bereitstellen, können wir beobachten und bestätigen, dass das LLM tatsächlich damit interagieren kann. Diese grundlegende Interaktion dient als grundlegender Schritt und demonstriert die grundlegenden Fähigkeiten des LLM, Befehle auszuführen. Der folgende Code erstellt ein LLM, wie wir es zuvor getan haben, sodass wir auf dieser grundlegenden Interaktion aufbauen und schrittweise komplexere Tools und Funktionen integrieren können.

In [ ]:
from langchain_openai import ChatOpenAI

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )



Im bereitgestellten Code erstellen wir ein einfaches Agent-Tool, das zum Drücken einer Taste entwickelt wurde. Wir beginnen mit dem Importieren der Tool-Klasse aus dem LangChain-Agentenmodul. Die Kernfunktionalität ist in der Funktion press_button definiert, die ein einzelnes Argument, value, annimmt. Wenn diese Funktion aufgerufen wird, druckt sie den bereitgestellten Wert und gibt die Zeichenfolge „Die Taste leuchtet rot!“ zurück.

Als nächstes instanziieren wir ein Tool-Objekt namens button_tool. Wir geben seinen Namen als „button_machine“ an und geben eine Beschreibung an, die seinen Zweck erklärt: Es ist ein großer roter Knopf, der gedrückt werden kann und bei entsprechender Aufforderung einen einzelnen Wert akzeptiert. Der Parameter func wird auf unsere press_button-Funktion gesetzt, wodurch die Aktion des Tools mit unserem definierten Verhalten verknüpft wird. Dieses Setup ermöglicht es dem LLM, mit dem Knopf-Tool zu interagieren, indem es es drückt und die entsprechende Reaktion beobachtet.

In [ ]:
from langchain.agents import Tool

def press_button(value):
print(f"*************{Wert}")
  return "The button glows red!"

button_tool = Tool(
    name="button_machine",
    description="A big red button that you can push. You can send a single value to the button, if asked. ",
    func=press_button,
)

In diesem Code konstruieren wir einen Agenten und weisen ihn an, die zuvor definierte Taste zu drücken. Wir beginnen mit dem Importieren der erforderlichen Module und Funktionen, einschließlich Hub von LangChain, create_tool_calling_agent und AgentExecutor. Anschließend ziehen wir eine Eingabeaufforderung aus dem LangChain-Hub, die als Vorlage für die Interaktionen unseres Agenten dient.

Wir definieren eine Liste mit dem Namen Tools, die unser zuvor erstelltes Buttontool enthält. Mithilfe der Funktion create_tool_calling_agent erstellen wir einen Agenten, der die von uns angegebenen Tools verwenden kann, geleitet von der Pulled-Eingabeaufforderung. Die Klasse AgentExecutor wird dann verwendet, um einen Agent_Executor zu instanziieren, der den Agenten mit den Tools verknüpft und eine ausführliche Ausgabe für einen besseren Einblick in die Interaktionen ermöglicht.

Zum Schluss rufen wir den Agenten mit dem Eingabebefehl auf: „Drücken Sie den großen roten Knopf und sagen Sie mir, was passiert.“ Dieser Befehl weist den Agenten an, das Knopfwerkzeug zu verwenden, und demonstriert so die Fähigkeit des Agenten, die definierte Aktion auszuführen und Feedback zum Ergebnis zu geben.

In [ ]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [button_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "Push the big red button, tell me what happens."})

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:5515: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))




> Entering new AgentExecutor chain...

Invoking: `button_machine` with `{'config': {'tags': ['big red button'], 'run_name': 'Push the Big Red Button'}}`


*************{'tags': ['big red button'], 'run_name': 'Push the Big Red Button'}
The button glows red!When you push the big red button, it glows red!

> Finished chain.


{'input': 'Push the big red button, tell me what happens.',
 'output': 'When you push the big red button, it glows red!'}

## Einen einfachen Roboter mit Werkzeugen steuern


In diesem Code erstellen wir ein komplexeres Tool mit drei Schaltflächen, die die Bewegung eines kleinen Autos steuern. Wir definieren eine CarTool-Klasse, die die Position (x, y) und Richtung (Nord, Ost, Süd, West) des Autos initialisiert. Die Klasse erstellt außerdem ein Tool-Objekt namens control_panel mit einer Beschreibung, die die Funktionen der drei Schaltflächen erklärt: rot, grün und gelb.

Die Klasse CarTool enthält Methoden, um das Auto vorwärts zu bewegen (move_forward), nach links abzubiegen (turn_left) und nach rechts abzubiegen (turn_right). Die Methode press_button verarbeitet Tastendrücke nach Farbe. Durch Drücken der roten Taste bewegt sich das Auto eine Einheit in die aktuelle Richtung vorwärts. Durch Drücken der grünen Taste dreht sich das Auto um 90 Grad nach links und durch Drücken der gelben Taste dreht sich das Auto um 90 Grad nach rechts. Wenn eine ungültige Tastenfarbe gedrückt wird, wird eine Fehlermeldung zurückgegeben.

Darüber hinaus liefert die Methode get_position die aktuelle Position und Richtung des Autos. Das Anwendungsbeispiel am Ende zeigt, wie man mit dem CarTool das Auto durch Drücken verschiedener Tasten und die Verarbeitung ungültiger Eingaben steuern kann.

In [ ]:
from langchain.tools import StructuredTool
from pydantic import BaseModel
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# Definieren Sie ein Pydantic-Schema für die Tastendruckeingabe
class ButtonPressInput(BaseModel):
    button_color: str

class CarTool:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.direction = 'north'
        self.directions = ['north', 'east', 'south', 'west']
        self.tool = StructuredTool(
            name="control_panel",
            description="""Three buttons, red, green and yellow that you can push.
            The red button moves you forward 1 unit.
            The green button turns left 90 degrees,
            the yellow button turns right 90 degrees.""",
            func=self.press_button,
            args_schema=ButtonPressInput  # Definieren Sie das erwartete Eingabeschema
        )

    def move_forward(self):
        if self.direction == 'north':
            self.y += 1
        elif self.direction == 'east':
            self.x += 1
        elif self.direction == 'south':
            self.y -= 1
        elif self.direction == 'west':
            self.x -= 1

    def turn_left(self):
        current_index = self.directions.index(self.direction)
        self.direction = self.directions[(current_index - 1) % 4]

    def turn_right(self):
        current_index = self.directions.index(self.direction)
        self.direction = self.directions[(current_index + 1) % 4]

    def press_button(self, button_color: str):
        button_color = button_color.strip().lower()
        if button_color == 'red':
            self.move_forward()
            result = "The button glows red, you move forward one unit."
        elif button_color == 'green':
            self.turn_left()
            result = "The button glows green, you turn 90 degrees to the left."
        elif button_color == 'yellow':
            self.turn_right()
            result = "The button glows yellow, you turn 90 degrees to the right."
        else:
            result = "The button buzzes, error."

print(f"Aktuelle Fahrzeugposition: {self.get_position()}")
        return result

    def get_position(self):
        return self.x, self.y, self.direction

# Beispielverwendung
car_tool = CarTool()

try:
    print(car_tool.tool.func('red'))  # Vorwärts gehen
    print(car_tool.tool.func('green'))  # Biegen Sie links ab
    print(car_tool.tool.func('yellow'))  # Biegen Sie rechts ab
    print(car_tool.tool.func('blue'))  # Ungültige Schaltflächenfarbe, löst ValueError aus
except ValueError as e:
    print(e)


Current car position: (0, 1, 'north')
The button glows red, you move forward one unit.
Current car position: (0, 1, 'west')
The button glows green, you turn 90 degrees to the left.
Current car position: (0, 1, 'north')
The button glows yellow, you turn 90 degrees to the right.
Current car position: (0, 1, 'north')
The button buzzes, error.


Wir konstruieren nun einen Agenten und stellen diesem Werkzeug die Aufgabe zu, das Auto aufzufordern, sich in einem Rechteck zu bewegen.

In [ ]:
# LLM initialisieren und einen Agenten erstellen
MODEL = 'gpt-4o-mini'
llm = ChatOpenAI(model=MODEL, temperature=0.2, n=1)

prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [car_tool.tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Rufen Sie den Agenten auf, um das Auto zu steuern
agent_executor.invoke({"input": "Push the buttons in a way that causes the car to move in a rectangle."})




/usr/local/lib/python3.10/dist-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

Invoking: `control_panel` with `{'button_color': 'red'}`


Current car position: (0, 2, 'north')
The button glows red, you move forward one unit.
Invoking: `control_panel` with `{'button_color': 'green'}`


Current car position: (0, 2, 'west')
The button glows green, you turn 90 degrees to the left.
Invoking: `control_panel` with `{'button_color': 'red'}`


Current car position: (-1, 2, 'west')
The button glows red, you move forward one unit.
Invoking: `control_panel` with `{'button_color': 'green'}`


Current car position: (-1, 2, 'south')
The button glows green, you turn 90 degrees to the left.
Invoking: `control_panel` with `{'button_color': 'red'}`


Current car position: (-1, 1, 'south')
The button glows red, you move forward one unit.
Invoking: `control_panel` with `{'button_color': 'green'}`


Current car position: (-1, 1, 'east')
The button glows green, you turn 90 degrees to the left.
Invoking: `control_panel` with `{'button_color': 're

{'input': 'Push the buttons in a way that causes the car to move in a rectangle.',
 'output': 'The car has moved in a rectangle as follows:\n\n1. Moved forward 1 unit.\n2. Turned 90 degrees to the left.\n3. Moved forward 1 unit.\n4. Turned 90 degrees to the left.\n5. Moved forward 1 unit.\n6. Turned 90 degrees to the left.\n7. Moved forward 1 unit.\n8. Turned 90 degrees to the left.\n\nThe car is now back at its starting position, having completed a rectangular path.'}